# Laboratorium 5 (4 pkt)

Celem czwartego laboratorium jest zapoznanie się oraz zaimplementowanie algorytmów głębokiego uczenia aktywnego. Zaimplementowane algorytmy będą testowane z wykorzystaniem środowiska z OpenAI - *CartPole*.


Dołączenie standardowych bibliotek

In [1]:
from collections import deque
import gym
import numpy as np
import random
from tqdm import tqdm

Dołączenie bibliotek do obsługi sieci neuronowych

In [2]:
# import tensorflow as tf
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras import Sequential
# import numpy as np

In [3]:
import torch as T
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

## Zadanie 1 - Double Deep Q-Network

<p style='text-align: justify;'>
Celem ćwiczenie jest zaimplementowanie algorytmu Double Deep Q-Network. Wartoscią oczekiwaną sieci jest:
\begin{equation}
       Q^*(s, a) \approx r + \gamma argmax_{a'}Q_\theta'(s', a') 
\end{equation}
a wagi pomiędzy sieciami wymieniane są co dziesięć aktualizacji wag sieci sterującej poczynaniami agenta ($Q$).
</p>

In [4]:
class ReplayBuffer(object):
    def __init__(self, mem_size, state_shape):
        self.mem_size = mem_size
        self.mem_cntr = 0
        self.state_memory = np.zeros((self.mem_size, state_shape), dtype=np.float32)
        self.new_state_memory = np.zeros((self.mem_size, state_shape), dtype=np.float32)
        self.action_memory = np.zeros(self.mem_size, dtype=np.int32)
        self.reward_memory = np.zeros(self.mem_size, dtype=np.float32)
        self.terminal_memory = np.zeros(self.mem_size, dtype=bool)

    def store_transition(self, state, action, reward, _state, done):
        index = self.mem_cntr % self.mem_size
        self.state_memory[index] = state
        self.new_state_memory[index] = _state
        self.action_memory[index] = action
        self.reward_memory[index] = reward
        self.terminal_memory[index] = 1 - done
        self.mem_cntr += 1

    def sample_buffer(self, batch_size):
        max_mem = min(self.mem_cntr, self.mem_size)

        batch = np.random.choice(max_mem, batch_size)

        states = self.state_memory[batch]
        actions = self.action_memory[batch]
        rewards = self.reward_memory[batch]
        _states = self.new_state_memory[batch]
        done = self.terminal_memory[batch]

        return states, actions, rewards, _states, done

In [5]:
class DQN(nn.Module):
    def __init__(self, lr, state_shape, n_actions, fc1, fc2):
        super(DQN, self).__init__()

        self.fc1 = nn.Linear(state_shape, fc1)
        self.fc2 = nn.Linear(fc1, fc2)
        self.output = nn.Linear(fc2, n_actions)
        self.optimizer = optim.Adam(self.parameters(), lr=lr)
        self.loss = nn.MSELoss()
        self.device = T.device('cuda:0')
        self.to(self.device)

    def forward(self, state):
        state = F.relu(self.fc1(state))
        state = F.relu(self.fc2(state))
        actions = self.output(state)

        return actions

In [6]:
class DDQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = ReplayBuffer(1000000, state_size)
        self.gamma = 0.95    # discount rate
        self.epsilon = 0.5  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.0005
        self.alpha = 0.001
        self.q = DQN(learning_rate, state_size, action_size, 64, 64)
        self.q_target = DQN(learning_rate, state_size, action_size, 64, 64)
        self.update_weights()
        self.evaluate=False

    # def _build_model(self):
    #     model = Sequential()
    #     model.add(Dense(128, input_shape=(self.state_size,), activation='relu'))
    #     model.add(Dense(128, activation='relu'))
    #     model.add(Dense(self.action_size, activation=None))
    #     model.compile(optimizer=Adam(learning_rate=self.alpha), loss='mse')
    #
    #     return model

    def remember(self, state, action, reward, _state, done):
        self.memory.store_transition(state, action, reward, _state, done)

    def get_action(self, state):
        if np.random.random() <= self.epsilon and not self.evaluate:
            action = np.random.choice(self.action_size)
        else:
            state = T.tensor(state).to(self.q.device)
            actions = self.q.forward(state)
            action = T.argmax(actions).item()

        return action


    def get_best_action(self, state):
        state = T.tensor(state).to(self.q.device)
        actions = self.q.forward(state)
        action = T.argmax(actions).item()

        return action

    def replay(self, batch_size):
        """
        Function learn network using randomly selected actions from the memory.
        First calculates Q value for the next state and choose action with the biggest value.
        Target value is calculated according to:
                Q(s,a) := (r + gamma * max_a(Q(s', a)))
        except the situation when the next action is the last action, in such case Q(s, a) := r.
        In order to change only those weights responsible for chosing given action, the rest values should be those
        returned by the network for state state.
        The network should be trained on batch_size samples.
        After each 10 Q Network trainings parameters should be copied to the target Q Network
        """
        if self.memory.mem_cntr < batch_size:
            return

        states, actions, rewards, _states, done = self.memory.sample_buffer(batch_size)

        states = T.tensor(states).to(self.q.device)
        _states = T.tensor(_states).to(self.q_target.device)

        q_pred = self.q(states)
        q_next = self.q_target(_states).cpu().detach().numpy()
        q_target = q_pred.cpu().detach().numpy()

        max_actions = np.argmax(q_target, axis=1)

        batch_index = np.arange(batch_size, dtype=np.int32)

        q_target[batch_index, actions] = rewards + self.gamma * q_next[batch_index, max_actions.astype(int)] * (1-done)
        q_target = T.tensor(q_target).to(self.q.device)

        loss = self.q.loss(q_pred, q_target).to(self.q.device)
        loss.backward()
        self.q.optimizer.step()

        if not self.memory.mem_cntr % 10:
            self.update_weights()

    def update_epsilon_value(self):
        self.epsilon = self.epsilon - self.epsilon_decay if self.epsilon > self.epsilon_min else self.epsilon_min

    def update_weights(self):
        self.q_target.load_state_dict(self.q.state_dict())


Czas przygotować model sieci, która będzie się uczyła działania w środowisku [*CartPool*](https://gym.openai.com/envs/CartPole-v0/):

In [7]:
env = gym.make("CartPole-v0")
state_size = env.observation_space.sample().shape[0]
action_size = env.action_space.n
learning_rate = 0.001

Czas nauczyć agenta gry w środowisku *CartPool*:

In [9]:
agent = DDQNAgent(state_size, action_size)

agent.epsilon = 0.75

done = False
batch_size = 64
EPISODES = 1000
counter = 0
for e in range(EPISODES):
    summary = []
    for _ in tqdm(range(100), desc=f'Epoch: {e}'):
        total_reward = 0
        state = env.reset()
        state = np.array(state, dtype=np.float32)

        for time in range(500):
            action = agent.get_action(state)
            _state, reward, done, _ = env.step(action)
            _state = np.array(_state, dtype=np.float32)
            total_reward += reward

            agent.remember(state, action, reward, _state, done)
            agent.replay(batch_size)
            state = _state
            if done:
                break

        agent.update_epsilon_value()
        summary.append(total_reward)
        
    print("epoch #{}\tmean reward = {:.3f}\tepsilon = {:.3f}".format(e, np.mean(summary), agent.epsilon))    
    
    if np.mean(summary) > 195:
        print ("You Win!")
        break


Epoch: 0: 100%|██████████| 100/100 [00:05<00:00, 19.08it/s]


epoch #0	mean reward = 19.080	epsilon = 0.700


Epoch: 1: 100%|██████████| 100/100 [00:04<00:00, 21.56it/s]


epoch #1	mean reward = 15.850	epsilon = 0.650


Epoch: 2: 100%|██████████| 100/100 [00:04<00:00, 20.74it/s]


...

epoch #90	mean reward = 9.370	epsilon = 0.010


Epoch: 91: 100%|██████████| 100/100 [00:03<00:00, 32.01it/s]


epoch #91	mean reward = 9.470	epsilon = 0.010


Epoch: 92:  20%|██        | 20/100 [00:00<00:02, 34.16it/s]


KeyboardInterrupt: 